# Ensemble Model Training - Simplified Single-Branch Models

This notebook trains separate simplified CNN models for each feature type and then creates an ensemble.

In [18]:
from datetime import datetime
import os
import json
import tensorflow as tf
import numpy as np
import pandas as pd
from keras.utils import to_categorical, Sequence
from keras.models import Model
from keras.layers import (
    Input, Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
)
from keras.callbacks import EarlyStopping
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.notebook import tqdm
from modules.PostgresDBHandler import PostgresDBHandler
from tqdm import tqdm
from tensorflow.keras.optimizers import Adam
from tensorflow import keras

In [2]:
# Configuration
dbParams = {
    "dbname": "mydatabase",
    "user": "myuser",
    "password": "mypassword",
    "host": "postgres_server",
    "port": "5432",
}

EPOCHS = 200
BATCH_SIZE = 32
KFOLD_SPLITS = 5
FIXED_LENGTH = 128

# Feature types to train models for
FEATURE_TYPES = [
    'mel_spectrogram', 'mfcc', 'chromagram', 'spectral_contrast',
    'tonnetz', 'constant_q', 'cqt', 'stft', 'harmonic_percussive', 'onset_strength'
]

# GPU configuration
gpus = tf.config.experimental.list_physical_devices("GPU")
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        print(f"Number of available GPUs: {len(gpus)}")
    except RuntimeError as e:
        print(e)

2025-07-07 19:25:17.370446: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2025-07-07 19:25:17.450665: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2256] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [3]:
# Initialize database connection
db = PostgresDBHandler(**dbParams)
db.connect()

# Get instrument mappings
instruments_mappings = db.get_mappings_instruments()
num_classes = len(instruments_mappings)
print(f"Number of instrument classes: {num_classes}")
print("Instruments:", instruments_mappings['name'].tolist())

db.close()

Number of instrument classes: 9
Instruments: ['violin', 'bass', 'flute', 'piccolo', 'trumpet', 'oboe', 'sax', 'clarinet', 'cello']


In [4]:
dbConnect = PostgresDBHandler(**dbParams)
dbConnect.connect()
audioIDs = dbConnect.get_all_unique_audio_ids_in_processed()
processed_data = dbConnect.get_processed_fit_data(audioIDs)

all_processed_data = []
for audio_id in audioIDs:
    features = dbConnect.get_all_feature_types_for_audio(audio_id)
    feature_dict = {f['featureTypeName']: f['featurePath'] for f in features}
    instrumentID = dbConnect.get_audio_file(audio_id)['instrumentID']
    feature_dict['instrumentID'] = instrumentID
    all_processed_data.append(feature_dict)

dbConnect.close()

In [5]:
processed_df = pd.DataFrame(all_processed_data)
processed_df

,mel_spectrogram,mfcc,chromagram,spectral_contrast,tonnetz,constant_q,cqt,stft,harmonic_percussive,onset_strength,instrumentID
0,ensemble_intermediate_results/mel_spectrogram/...,ensemble_intermediate_results/mfcc/b4eff491-ba...,ensemble_intermediate_results/chromagram/97320...,ensemble_intermediate_results/spectral_contras...,ensemble_intermediate_results/tonnetz/edf38d8e...,ensemble_intermediate_results/constant_q/c57fd...,ensemble_intermediate_results/cqt/cdee10e8-02c...,ensemble_intermediate_results/stft/fd3e6642-db...,ensemble_intermediate_results/harmonic_percuss...,ensemble_intermediate_results/onset_strength/7...,1
1,ensemble_intermediate_results/mel_spectrogram/...,ensemble_intermediate_results/mfcc/cb7069c3-eb...,ensemble_intermediate_results/chromagram/56820...,ensemble_intermediate_results/spectral_contras...,ensemble_intermediate_results/tonnetz/e8615bcb...,ensemble_intermediate_results/constant_q/2800b...,ensemble_intermediate_results/cqt/f9a1e89d-2c5...,ensemble_intermediate_results/stft/b12a54e3-cb...,ensemble_intermediate_results/harmonic_percuss...,ensemble_intermediate_results/onset_strength/3...,5
2,ensemble_intermediate_results/mel_spectrogram/...,ensemble_intermediate_results/mfcc/e320d53d-b3...,ensemble_intermediate_results/chromagram/74786...,ensemble_intermediate_results/spectral_contras...,ensemble_intermediate_results/tonnetz/0566b804...,ensemble_intermediate_results/constant_q/c415e...,ensemble_intermediate_results/cqt/7fbfb24e-12b...,ensemble_intermediate_results/stft/9cdd13f6-47...,ensemble_intermediate_results/harmonic_percuss...,ensemble_intermediate_results/onset_strength/b...,4
3,ensemble_intermediate_results/mel_spectrogram/...,ensemble_intermediate_results/mfcc/30b73223-05...,ensemble_intermediate_results/chromagram/05881...,ensemble_intermediate_results/spectral_contras...,ensemble_intermediate_results/tonnetz/3d60483b...,ensemble_intermediate_results/constant_q/0bc74...,ensemble_intermediate_results/cqt/b7062127-882...,ensemble_intermediate_results/stft/bb17e38a-b3...,ensemble_intermediate_results/harmonic_percuss...,ensemble_intermediate_results/onset_strength/2...,6
4,ensemble_intermediate_results/mel_spectrogram/...,ensemble_intermediate_results/mfcc/950e2465-b2...,ensemble_intermediate_results/chromagram/a733d...,ensemble_intermediate_results/spectral_contras...,ensemble_intermediate_results/tonnetz/8bd82a0f...,ensemble_intermediate_results/constant_q/13fab...,ensemble_intermediate_results/cqt/7e056e87-607...,ensemble_intermediate_results/stft/c3e18b59-fb...,ensemble_intermediate_results/harmonic_percuss...,ensemble_intermediate_results/onset_strength/d...,4
...,...,...,...,...,...,...,...,...,...,...,...
895,ensemble_intermediate_results/mel_spectrogram/...,ensemble_intermediate_results/mfcc/5c041df7-89...,ensemble_intermediate_results/chromagram/6df3f...,ensemble_intermediate_results/spectral_contras...,ensemble_intermediate_results/tonnetz/a175304c...,ensemble_intermediate_results/constant_q/5d998...,ensemble_intermediate_results/cqt/d0aa2576-88d...,ensemble_intermediate_results/stft/5cefb2c7-fb...,ensemble_intermediate_results/harmonic_percuss...,ensemble_intermediate_results/onset_strength/e...,3
896,ensemble_intermediate_results/mel_spectrogram/...,ensemble_intermediate_results/mfcc/b8ae91b6-dd...,ensemble_intermediate_results/chromagram/5054d...,ensemble_intermediate_results/spectral_contras...,ensemble_intermediate_results/tonnetz/5c61d6ce...,ensemble_intermediate_results/constant_q/4aa61...,ensemble_intermediate_results/cqt/e0780fab-3a3...,ensemble_intermediate_results/stft/b519d38c-d6...,ensemble_intermediate_results/harmonic_percuss...,ensemble_intermediate_results/onset_strength/f...,7
897,ensemble_intermediate_results/mel_spectrogram/...,ensemble_intermediate_results/mfcc/a3366792-9c...,ensemble_intermediate_results/chromagram/af45a...,ensemble_intermediate_results/spectral_contras...,ensemble_intermediate_results/tonnetz/0cfb6ceb...,ensemble_intermediate_results/constant_

In [6]:
def get_input_shape(feature_type, df):
    feature_path_col = feature_type 
    for path in df[feature_path_col]:
        if isinstance(path, str) and os.path.exists(path):
            arr = np.load(path)
            return arr.shape
    raise ValueError(f"No valid file found for {feature_type}")

In [7]:
class SingleFeatureDataGenerator(Sequence):
    def __init__(self, df, feature_col, batch_size=32, shuffle=True, num_classes=None):
        self.df = df.reset_index(drop=True)
        self.feature_col = feature_col
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.num_classes = num_classes
        self.on_epoch_end()

    def __len__(self):
        return int(np.ceil(len(self.df) / self.batch_size))

    def on_epoch_end(self):
        self.indices = np.arange(len(self.df))
        if self.shuffle:
            np.random.shuffle(self.indices)

    def __getitem__(self, index):
        batch_indices = self.indices[index * self.batch_size:(index + 1) * self.batch_size]
        batch_df = self.df.iloc[batch_indices]

        X = []
        y = []

        for _, row in batch_df.iterrows():
            try:
                arr = np.load(row[self.feature_col])
            except Exception as e:
                print(f"Error loading {row[self.feature_col]}: {e}")
                continue
        
            if np.isnan(arr).any() or np.isinf(arr).any():
                raise ValueError(f"Feature file {row[self.feature_col]} contains NaNs or Infs.")
        
            arr = (arr - np.mean(arr)) / (np.std(arr) + 1e-8)
            if arr.ndim == 2:
                arr = np.expand_dims(arr, -1)  # shape: (H, W, 1)
        
            X.append(arr)
            y.append(row['instrumentID'])  # already label-encoded
        
        X = np.array(X)
        y = to_categorical(np.array(y), num_classes=self.num_classes)
        
        return X, y

In [14]:
def create_simple_model(input_shape, num_classes, model_name="simple_cnn"):
    input_layer = Input(shape=(*input_shape, 1), name=f"{model_name}_input")

    x = Conv2D(4, (3, 3), activation='relu', padding='same')(input_layer)
    x = BatchNormalization()(x)

    x = Flatten()(x)

    output = Dense(num_classes, activation='softmax', name=f"{model_name}_output")(x)

    model = Model(inputs=input_layer, outputs=output, name=model_name)
    return model

In [16]:
results = {}

for feature_type in tqdm(FEATURE_TYPES, desc="Training features"):
    print(f"\n{'='*40}\nTraining model for {feature_type}\n{'='*40}")

    feature_col = feature_type
    feature_df = processed_df.dropna(subset=[feature_col])
    
    # Global label encoder
    label_encoder = LabelEncoder()
    label_encoder.fit(feature_df['instrumentID'])

    feature_df = feature_df.copy()
    feature_df['instrumentID'] = label_encoder.transform(feature_df['instrumentID'])
    num_classes = len(label_encoder.classes_)
    input_shape = get_input_shape(feature_type, feature_df)


    kf = KFold(n_splits=KFOLD_SPLITS, shuffle=True, random_state=42)
    accuracy_list, loss_list, history_list = [], [], []
    classification_reports, confusion_matrices = [], []

    for fold, (train_idx, test_idx) in enumerate(kf.split(feature_df)):
        print(f"\n--- Fold {fold+1}/{KFOLD_SPLITS} ---")
        train_df = feature_df.iloc[train_idx].reset_index(drop=True)
        test_df = feature_df.iloc[test_idx].reset_index(drop=True)

        train_df, val_df = train_test_split(
            train_df, test_size=0.2, random_state=42, stratify=train_df['instrumentID'])

        # Generators (labels are already encoded)
        train_gen = SingleFeatureDataGenerator(train_df, feature_col, BATCH_SIZE, shuffle=True, num_classes=num_classes)
        val_gen   = SingleFeatureDataGenerator(val_df,   feature_col, BATCH_SIZE, shuffle=False, num_classes=num_classes)
        test_gen  = SingleFeatureDataGenerator(test_df,  feature_col, BATCH_SIZE, shuffle=False, num_classes=num_classes)

        # Model
        model = create_simple_model(input_shape, num_classes, model_name=feature_type)
        model.compile(optimizer=Adam(learning_rate=0.01), loss='categorical_crossentropy', metrics=['accuracy'])

        early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

        history = model.fit(train_gen, validation_data=val_gen, epochs=EPOCHS, callbacks=[early_stopping])
        history_list.append(history.history)

        # Evaluation
        loss, acc = model.evaluate(test_gen)
        loss_list.append(loss)
        accuracy_list.append(acc)
        print(f"{feature_type} - Fold {fold+1} Test accuracy: {acc:.4f}")

        # Predictions & Reports
        y_pred = model.predict(test_gen)
        y_pred_classes = np.argmax(y_pred, axis=1)
        y_true = []
        for _, labels in test_gen:
            y_true.extend(np.argmax(labels, axis=1))
        y_true = np.array(y_true)

        report = classification_report(y_true, y_pred_classes, output_dict=True)
        classification_reports.append(report)
        conf_matrix = confusion_matrix(y_true, y_pred_classes).tolist()
        confusion_matrices.append(conf_matrix)

        # Save model
        os.makedirs(f"models/{feature_type}", exist_ok=True)
        model.save(f"models/{feature_type}/model_fold{fold+1}.keras")

    # Save results
    results[feature_type] = {
        "accuracy_list": accuracy_list,
        "loss_list": loss_list,
        "histories": history_list,
        "classification_reports": classification_reports,
        "confusion_matrices": confusion_matrices,
    }

    with open(f"models/{feature_type}/results.json", "w") as f:
        json.dump(results[feature_type], f, indent=2)

print("\nAll training complete. Models and results saved in 'models/'")

Training features:   0%|          | 0/10 [00:00<?, ?it/s]


Training model for mel_spectrogram

--- Fold 1/5 ---
Epoch 1/200
18/18 [==============================] - 0s 12ms/step - loss: 46.9619 - accuracy: 0.4427 - val_loss: 41.0709 - val_accuracy: 0.5139
Epoch 2/200
18/18 [==============================] - 0s 8ms/step - loss: 14.0214 - accuracy: 0.7795 - val_loss: 35.3122 - val_accuracy: 0.5417
Epoch 3/200
18/18 [==============================] - 0s 8ms/step - loss: 3.8550 - accuracy: 0.9045 - val_loss: 30.9735 - val_accuracy: 0.5833
Epoch 4/200
18/18 [==============================] - 0s 8ms/step - loss: 1.1915 - accuracy: 0.9479 - val_loss: 23.7811 - val_accuracy: 0.6667
Epoch 5/200
18/18 [==============================] - 0s 8ms/step - loss: 1.0495 - accuracy: 0.9583 - val_loss: 24.6466 - val_accuracy: 0.6181
Epoch 6/200
18/18 [==============================] - 0s 8ms/step - loss: 0.6943 - accuracy: 0.9601 - val_loss: 26.0121 - val_accuracy: 0.6042
Epoch 7/200
18/18 [==============================] - 0s 8ms/step - loss: 0.3958 - accuracy:

Training features:  10%|█         | 1/10 [00:42<06:23, 42.57s/it]


Training model for mfcc

--- Fold 1/5 ---
Epoch 1/200
18/18 [==============================] - 0s 10ms/step - loss: 10.9700 - accuracy: 0.3108 - val_loss: 4.2221 - val_accuracy: 0.2986
Epoch 2/200
18/18 [==============================] - 0s 7ms/step - loss: 2.5693 - accuracy: 0.6389 - val_loss: 4.4004 - val_accuracy: 0.3264
Epoch 3/200
18/18 [==============================] - 0s 7ms/step - loss: 1.0621 - accuracy: 0.7726 - val_loss: 5.6707 - val_accuracy: 0.2778
Epoch 4/200
18/18 [==============================] - 0s 7ms/step - loss: 0.6454 - accuracy: 0.8351 - val_loss: 5.3840 - val_accuracy: 0.2639
Epoch 5/200
18/18 [==============================] - 0s 7ms/step - loss: 0.4309 - accuracy: 0.8872 - val_loss: 3.4160 - val_accuracy: 0.3611
Epoch 6/200
18/18 [==============================] - 0s 7ms/step - loss: 0.3298 - accuracy: 0.9062 - val_loss: 3.1817 - val_accuracy: 0.4028
Epoch 7/200
18/18 [==============================] - 0s 7ms/step - loss: 0.2942 - accuracy: 0.9132 - val_loss

Training features:  20%|██        | 2/10 [01:07<04:16, 32.06s/it]

...


Training model for onset_strength

--- Fold 1/5 ---
Epoch 1/200
18/18 [==============================] - 0s 11ms/step - loss: 2.4003 - accuracy: 0.2622 - val_loss: 2.0904 - val_accuracy: 0.2153
Epoch 2/200
18/18 [==============================] - 0s 7ms/step - loss: 1.4128 - accuracy: 0.5295 - val_loss: 2.0831 - val_accuracy: 0.1736
Epoch 3/200
18/18 [==============================] - 0s 7ms/step - loss: 1.1419 - accuracy: 0.6024 - val_loss: 2.0245 - val_accuracy: 0.2500
Epoch 4/200
18/18 [==============================] - 0s 7ms/step - loss: 0.9534 - accuracy: 0.6649 - val_loss: 2.0296 - val_accuracy: 0.2639
Epoch 5/200
18/18 [==============================] - 0s 7ms/step - loss: 0.7966 - accuracy: 0.7378 - val_loss: 2.0397 - val_accuracy: 0.2361
Epoch 6/200
18/18 [==============================] - 0s 7ms/step - loss: 0.7187 - accuracy: 0.7639 - val_loss: 2.0057 - val_accuracy: 0.2569
Epoch 7/200
18/18 [==============================] - 0s 7ms/step - loss: 0.6287 - accuracy: 0.7778 -

/opt/conda/envs/myenv/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/envs/myenv/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/envs/myenv/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


18/18 [==============================] - 0s 10ms/step - loss: 2.4999 - accuracy: 0.2396 - val_loss: 2.1143 - val_accuracy: 0.1944
Epoch 2/200
18/18 [==============================] - 0s 7ms/step - loss: 1.7695 - accuracy: 0.3785 - val_loss: 2.0485 - val_accuracy: 0.2431
Epoch 3/200
18/18 [==============================] - 0s 7ms/step - loss: 1.4914 - accuracy: 0.4444 - val_loss: 2.0896 - val_accuracy: 0.1319
Epoch 4/200
18/18 [==============================] - 0s 7ms/step - loss: 1.3497 - accuracy: 0.5434 - val_loss: 2.1453 - val_accuracy: 0.1319
Epoch 5/200
18/18 [==============================] - 0s 7ms/step - loss: 1.3097 - accuracy: 0.5382 - val_loss: 2.1882 - val_accuracy: 0.1250
Epoch 6/200
18/18 [==============================] - 0s 6ms/step - loss: 1.2236 - accuracy: 0.5694 - val_loss: 2.1214 - val_accuracy: 0.1806
Epoch 7/200
18/18 [==============================] - 0s 7ms/step - loss: 1.0993 - accuracy: 0.6250 - val_loss: 1.9800 - val_accuracy: 0.2222
Epoch 8/200
18/18 [=====

Training features: 100%|██████████| 10/10 [08:38<00:00, 51.84s/it]


All training complete. Models and results saved in 'models/'


In [24]:
print("\n" + "="*50)
print("Creating Ensemble Predictions")
print("="*50)

processed_df['instrumentID'] = processed_df['instrumentID'] - 1

ensemble_accuracies = []
ensemble_reports = []
ensemble_conf_matrices = []

for fold in range(KFOLD_SPLITS):
    print(f"\n--- Ensemble Fold {fold + 1}/{KFOLD_SPLITS} ---")
    fold_preds = []
    y_true = None

    for feature_type in FEATURE_TYPES:
        # Load model for this fold
        model_path = f"models/{feature_type}/model_fold{fold+1}.keras"
        if not os.path.exists(model_path):
            print(f"Model not found: {model_path}")
            continue
        model = keras.models.load_model(model_path)

        # Get test data for this fold
        feature_df = processed_df[[feature_type, 'instrumentID']].dropna().reset_index(drop=True)
        kf = KFold(n_splits=KFOLD_SPLITS, shuffle=True, random_state=42)
        train_idx, test_idx = list(kf.split(feature_df))[fold]
        test_df = feature_df.iloc[test_idx].reset_index(drop=True)
        test_gen = SingleFeatureDataGenerator(test_df, feature_type, batch_size=BATCH_SIZE, shuffle=False, num_classes=num_classes)
        preds = model.predict(test_gen, verbose=0)
        fold_preds.append(preds)
        if y_true is None:
            # Get true labels from generator
            y_true = []
            for _, labels in test_gen:
                y_true.extend(np.argmax(labels, axis=1))
            y_true = np.array(y_true)

    if fold_preds:
        ensemble_pred = np.mean(fold_preds, axis=0)
        ensemble_pred_classes = np.argmax(ensemble_pred, axis=1)
        acc = accuracy_score(y_true, ensemble_pred_classes)
        ensemble_accuracies.append(acc)
        print(f"Ensemble Accuracy: {acc:.4f}")
        report = classification_report(y_true, ensemble_pred_classes, output_dict=True)
        ensemble_reports.append(report)
        conf_matrix = confusion_matrix(y_true, ensemble_pred_classes).tolist()
        ensemble_conf_matrices.append(conf_matrix)


Creating Ensemble Predictions

--- Ensemble Fold 1/5 ---
Ensemble Accuracy: 0.9611

--- Ensemble Fold 2/5 ---
Ensemble Accuracy: 0.9778

--- Ensemble Fold 3/5 ---
Ensemble Accuracy: 0.9611

--- Ensemble Fold 4/5 ---
Ensemble Accuracy: 0.9889

--- Ensemble Fold 5/5 ---
Ensemble Accuracy: 0.9222


In [25]:
os.makedirs("ensemble_results", exist_ok=True)
date_part = datetime.now().date().__str__().replace('-', '_')
results_path = os.path.join("ensemble_results", f"ensemble_results_{date_part}.json")
ensemble_results = {
    "accuracy_list": ensemble_accuracies,
    "classification_reports": ensemble_reports,
    "confusion_matrices": ensemble_conf_matrices,
}
with open(results_path, "w") as f:
    json.dump(ensemble_results, f, indent=2)
print(f"\nEnsemble results saved to: {results_path}")


Ensemble results saved to: ensemble_results/ensemble_results_2025_07_07.json


In [26]:
print("\n" + "="*60)
print("TRAINING SUMMARY")
print("="*60)

print("\nIndividual Model Performance:")
for feature_type in FEATURE_TYPES:
    if feature_type in results:
        accuracies = results[feature_type]['accuracy_list']
        mean_acc = np.mean(accuracies)
        std_acc = np.std(accuracies)
        print(f"  {feature_type}: {mean_acc:.4f} ± {std_acc:.4f}")

print("\nEnsemble Performance:")
ensemble_mean = np.mean(ensemble_accuracies)
ensemble_std = np.std(ensemble_accuracies)
print(f"  Ensemble: {ensemble_mean:.4f} ± {ensemble_std:.4f}")

# Find best individual model
best_individual = max(
    [(ft, np.mean(results[ft]['accuracy_list'])) for ft in FEATURE_TYPES if ft in results],
    key=lambda x: x[1]
)
improvement = ensemble_mean - best_individual[1]
print(f"\nBest Individual Model: {best_individual[0]} ({best_individual[1]:.4f})")
print(f"Ensemble Improvement: {improvement:.4f} ({improvement*100:.2f}%)")


TRAINING SUMMARY

Individual Model Performance:
  mel_spectrogram: 0.8878 ± 0.0432
  mfcc: 0.8767 ± 0.0206
  chromagram: 0.3956 ± 0.0373
  spectral_contrast: 0.5733 ± 0.0642
  tonnetz: 0.3300 ± 0.0428
  constant_q: 0.8733 ± 0.0231
  cqt: 0.8789 ± 0.0506
  stft: 0.6511 ± 0.0995
  harmonic_percussive: 0.9167 ± 0.0165
  onset_strength: 0.2144 ± 0.0303

Ensemble Performance:
  Ensemble: 0.9622 ± 0.0226

Best Individual Model: harmonic_percussive (0.9167)
Ensemble Improvement: 0.0456 (4.56%)
